In [1]:
from typing import Dict, List, TypeVar, Generic, Optional, Literal

from pydantic import BaseModel, Field

U = TypeVar("U")

class Coordinate(BaseModel):
    lat: Optional[float] = None
    lng: Optional[float] = None


class ReqUserId(BaseModel):
    user_id: str


class ReqModel(BaseModel, Generic[U]):
    message: str
    request_info: Dict
    request_body: U


class ReqCityCountry(BaseModel):
    city_name: str
    country_name: str

class ReqPrdcerLyrMapData(ReqUserId):
    prdcer_lyr_id: Optional[str] = ""


class ReqFetchDataset(ReqCityCountry, ReqPrdcerLyrMapData, Coordinate):
    boolean_query: Optional[str] = ""
    action: Optional[str] = ""
    page_token: Optional[str] = ""
    search_type: Optional[str] = "default"
    text_search: Optional[str] = ""
    zoom_level: Optional[int] = 0
    radius: Optional[float] = 30000.0
    _bounding_box: Optional[list[float]] = []
    _included_types: Optional[list[str]] = []
    _excluded_types: Optional[list[str]] = []


class ReqLLMFetchDataset(BaseModel):
    """Extract Location Based Information from the Query"""

    query: str = Field(
        default = "",
        description = "Original query passed by the user."
    )
    queryStatus: Literal["Valid", "Invalid"] = Field(
        default="Valid",
        description="Status of the query that depends on approved categories.It must be either 'Valid' or 'Invalid'"
    )
    message: str = Field(
        default = "",
        description = "Response message for the User after processing the query. It helps user to identify issues in the query"
    )
    requestStatus: Literal["Processed", "NotProcessed"] = Field(
        default="NotProcessed",
        description="Set to processed whenever an LLM encounters the query is processed by the LLM"
    )
    fetch_dataset_request: Optional[ReqFetchDataset] = Field(
        default=None,
        description="An object containing detailed request parameters for fetching dataset"
    )
    cost: str = Field(
        default = '',
        description = "The cost value returned by calculate_cost_tool"
    )

In [2]:
from typing import Dict, List, TypeVar, Generic, Literal, Any, Optional, Union

T = TypeVar("T")

class ResModel(BaseModel, Generic[T]):
    message: str
    request_id: str
    data: T


class ResLLMFetchDataset(BaseModel):
    """Extract Location Based Information from the Query"""

    query: str = Field(
        default = "",
        description = "Original query passed by the user."
    )
    queryStatus: Literal["Valid", "Invalid"] = Field(
        default="Valid",
        description="Status of the query that depends on approved categories.It must be either 'Valid' or 'Invalid'"
    )
    message: str = Field(
        default = "",
        description = "Response message for the User after processing the query. It helps user to identify issues in the query"
    )
    requestStatus: Literal["Processed", "NotProcessed"] = Field(
        default="NotProcessed",
        description="Set to processed whenever an LLM encounters the query is processed by the LLM"
    )
    fetch_dataset_request: Optional[ReqFetchDataset] = Field(
        default=None,
        description="An object containing detailed request parameters for fetching dataset"
    )
    cost: str = Field(
        default = '',
        description = "The cost value returned by calculate_cost_tool"
    )

In [16]:
data = {
  "query": "",
  "queryStatus": "Valid",
  "message": "",
  "requestStatus": "NotProcessed",
  "fetch_dataset_request": {
    "lat": 0,
    "lng": 0,
    "user_id": "string",
    "prdcer_lyr_id": "",
    "city_name": "string",
    "country_name": "string",
    "boolean_query": "",
    "action": "",
    "page_token": "",
    "search_type": "default",
    "text_search": "",
    "zoom_level": 0,
    "radius": 30000
  },
  "cost": ""
}


Request_body = ResLLMFetchDataset(**data)


In [17]:
Request_body

ResLLMFetchDataset(query='', queryStatus='Valid', message='', requestStatus='NotProcessed', fetch_dataset_request=ReqFetchDataset(lat=0.0, lng=0.0, user_id='string', prdcer_lyr_id='', city_name='string', country_name='string', boolean_query='', action='', page_token='', search_type='default', text_search='', zoom_level=0, radius=30000.0), cost='')

In [25]:
import requests

def fetch_approved_data(url: str):
    """
    Sends a GET request to the specified API endpoint.

    :param url: The API URL to call.
    :return: Response JSON or None if the request fails.
    """
    headers = {"accept": "application/json"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises an error for HTTP failures (4xx, 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return None  # Return None if the request fails

def extract_countries_and_cities(data):
    """
    Extracts separate lists of countries and cities from the given data.

    :param data: Dictionary containing countries as keys and city details as values.
    :return: Tuple (countries_list, cities_list)
    """
    if not data:  # Handle None or empty data safely
        return [], []

    countries = list(data.keys())  # Extract country names
    cities = [city["name"] for cities in data.values() for city in cities]  # Extract all city names

    return countries, cities

In [27]:
country_city_data = fetch_approved_data("http://37.27.195.216:8000/fastapi/country_city")
category_data = fetch_approved_data("http://37.27.195.216:8000/fastapi/nearby_categories")
if country_city_data and "data" in country_city_data:
    Approved_Countries, Approved_Cities = extract_countries_and_cities(country_city_data["data"])
else:
    Approved_Countries, Approved_Cities = [], []
    print("Warning: Failed to fetch approved countries and cities.")

if category_data and "data" in category_data:
    Approved_Categories = category_data["data"]
else:
    Approved_Categories = []
    print("Warning: Failed to fetch approved categories.")


In [31]:
system_message = """You are an intelligent assistant that extracts structured data for a location-based search API. Only process queries that specifically request 
                        information about places in a city or country. Add the country name automatically and try to be consistent.
                        
                        Reject queries that:
                        1. Do not explicitly mention searching for a place (e.g., "How to dance in Dubai" or "Weather in Paris").
                        2. Are general knowledge or instructional queries (e.g., "History of London" or "How to apply for a visa").
                        3. Contain inappropriate, offensive, illegal, or nonsensical requests.
                        4. Do not belong to the approved categories for places (e.g. tea shops is not an approved category)
                        5. Do not belong to approved cities and countries.
                        6. Contain multiple cities or countries
                        7. Do not contain a city name
                        
                        #Approved Categories for Places#
                        {Approved_Categories}
                        # Approved Countries #
                        {Approved_Countries}                     
                        # Approved Cities #
                        {Approved_Cities} 
                        
                        #Formation of Boolean Queries
                        1. Boolean Query is a string formed by joining all the places in the query by 'OR' / 'AND'
                        2. Examine the user query semantically and construct a Boolean Query



                    """

In [41]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, model_validator

model = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.0)

parser = PydanticOutputParser(pydantic_object=ResLLMFetchDataset)

prompt = PromptTemplate(
    template="{system_message}.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Find banks and pharmacies in Toronto","system_message":system_message})
outputResponse = parser.invoke(output)

In [42]:
outputResponse

ResLLMFetchDataset(query='Find banks and pharmacies in Toronto', queryStatus='Valid', message='', requestStatus='Processed', fetch_dataset_request=ReqFetchDataset(lat=None, lng=None, user_id='', prdcer_lyr_id='', city_name='Toronto', country_name='Canada', boolean_query='banks AND pharmacies', action='', page_token='', search_type='default', text_search='', zoom_level=0, radius=30000.0), cost='')

In [38]:
isinstance(outputResponse,ResLLMFetchDataset)

True

In [39]:
def cost_calculator():
    


SyntaxError: expected '(' (2810423022.py, line 1)

In [40]:
def calculate_cost(Request: ResLLMFetchDataset):
    api_requests = [Request.fetch_dataset_request]
    API_ENDPOINT = "http://37.27.195.216:8000/fastapi/cost_calculator"
    responses = []
    total_cost = 0
    
    for api_request in api_requests:
        try:
            payload = {
                "message": "Cost calculation request from LLM",
                "request_info": {},  # Add relevant request info if needed
                "request_body": api_request.dict()
            }
            
            response = requests.post(
                API_ENDPOINT,
                json=payload
            )
            response.raise_for_status()
            total_cost += response.json()["data"]["cost"]
            responses.append(response.json())
        except requests.exceptions.RequestException as e:
            responses.append({"error": f"API request failed for {api_request}: {e}"})
    Request.cost = str(total_cost)

    return Request

In [43]:
calculate_cost(outputResponse)

ResLLMFetchDataset(query='Find banks and pharmacies in Toronto', queryStatus='Valid', message='', requestStatus='Processed', fetch_dataset_request=ReqFetchDataset(lat=None, lng=None, user_id='', prdcer_lyr_id='', city_name='Toronto', country_name='Canada', boolean_query='banks AND pharmacies', action='', page_token='', search_type='default', text_search='', zoom_level=0, radius=30000.0), cost='0.4')

In [ ]:
def process_llm_query(Request:ReqLLMFetchDataset):
    model = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.0)

    parser = PydanticOutputParser(pydantic_object=ResLLMFetchDataset)
    
    prompt = PromptTemplate(
        template="{system_message}.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    # And a query intended to prompt a language model to populate the data structure.
    prompt_and_model = prompt | model
    output = prompt_and_model.invoke({"query": "Find banks and pharmacies in Toronto","system_message":system_message})
    outputResponse = parser.invoke(output)
    if outputResponse.fetch_dataset_request is None:
        return outputResponse
    else:
        return calculate_cost(outputResponse)
    